## Forest Health Threshold Update - Results

### Setup

In [ ]:
# import packages
import arcpy
import pandas as pd
from pathlib import Path
from utils import *
import plotly.express as px
import plotly.graph_objects as go
from arcgis import GeoAccessor, GeoSeriesAccessor
import pickle

# setup workspace variables
arcpy.env.workspace = r"F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\ForestHealth_ThresholdUpdate.gdb"
arcpy.env.overwriteOutput = True

#format paths
sdeBase   = Path("F:\GIS\DB_CONNECT\Vector.sde")
# workspace = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb")
downloads = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\SNV_RRK")
output    = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct")

# set style variables
template = 'plotly_white'
font     = 'Calibri'
# colors for tables
headerColor = '#6680a8'
rowColor = 'white'
lastrowColor = '#eeeeee'

In [ ]:
# list datasets in workspace
feature_data = arcpy.ListFeatureClasses()
raster_data  = arcpy.ListRasters()
table_data   = arcpy.ListTables()
# print feature classes, then rasters, then tables
print("Feature Classes:")
for fc in feature_data:
    print(fc)
print("\nRasters:")
for raster in raster_data:
    print(raster)
print("\nTables:")
for table in table_data:
    print(table)

## Stand Density

In [ ]:
# 
raster_path = "stand_density_threshold_assessment"

# Describe the raster and get the attribute table
fields = [f.name for f in arcpy.ListFields(raster_path)]
rows = arcpy.da.SearchCursor(raster_path, fields)

# Build DataFrame
data = [list(row) for row in rows]
df = pd.DataFrame(data, columns=fields)
df

In [ ]:
raster_path = "stand_density_threshold_assessment"
# Describe the raster and get the attribute table
fields = [f.name for f in arcpy.ListFields(raster_path)]
rows = arcpy.da.SearchCursor(raster_path, fields)

# Build DataFrame
data = [list(row) for row in rows]
df = pd.DataFrame(data, columns=fields)

# Apply the classification function to each ro
df["TargetStatus"] = df.apply(classify_snrrk_tpa, axis=1)
#group by target status and acres
df_grouped = df.groupby("TargetStatus")["Acres"].sum().reset_index()
# calculate percentage of acres in each target status
df_grouped["Percentage"] = (df_grouped["Acres"] / df_grouped["Acres"].sum()) * 100
# drop the Unknown category
df_grouped = df_grouped[df_grouped["TargetStatus"] != "Unknown"]
# format the percentage column
df_grouped["Percentage"] = df_grouped["Percentage"].round(1)

# get copy for chart
df = df_grouped.copy()
# bar chart of target status by acres
fig = px.bar(df, x="TargetStatus", y="Acres", 
            text=df['Percentage'].astype(str) + '%',
            title="Stand Density",
            color_discrete_sequence=["#ABCD66", "#728944"],
            custom_data=["TargetStatus"]
            )
fig.update_traces(marker_line_color='rgb(8,48,107)',
                    marker_line_width=0.1, opacity=0.9,
                    textfont_size=14, textposition="inside",
                    hovertemplate="<br>".join(
                        ["<b>%{y:,.0f} acres</b> of the forested area are", 
                            "<i>%{customdata[0]}</i> for resilience"]
                    )
                    + "<extra></extra>",
            )
fig.update_layout(title_x=0.5, title_font_size=20,
                    uniformtext_minsize=8, uniformtext_mode='hide',
                    title_font_color='#6680a8',
                    xaxis_title="Stand Density - SNRRK",
                    yaxis_title="Acres",
                    hovermode="x unified",
                    template=template,
                    legend=dict(title_text='',
                                orientation="h",
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1),
                    legend_title_text='',
                    font_family=font,
                    font_color='#6680a8',
                    plot_bgcolor='white',
                    showlegend=True
                        )
# no legendd
fig.update_traces(showlegend=False)
fig.show()
# save to HTML
fig.write_html(config = {"displayModeBar": False},
               file="DataVisualizations/StandDensity_Chart_SNRRK.html", 
               full_html= False, 
               include_plotlyjs='cdn')

## Seral Stage

In [ ]:
# stacked bar chart of seral stage by canopy cover class
df = dfSeralCanopy60_Tahoe.copy()

# classify seral stage by category Early Seral Opne - Early Seral, etch
df['SeralStage'] = df['Category'].replace(
    {'Early Seral Open':'Early Seral',
        'Early Seral Closed':'Early Seral',
        'Mid Seral Open':'Mid Seral',
        'Mid Seral Closed':'Mid Seral',
        'Late Seral Open':'Late Seral',
        'Late Seral Closed':'Late Seral'
    }
)

## get the percentage of acres in each category
df['Percentage'] = ((df['Acres'] / df['Acres'].sum()) * 100).round(1)

fig = px.bar(df, x="SeralStage", y="Acres", 
            color="Category",
            text=df['Percentage'].astype(str) + '%',
            title="Composition & Age",
            color_discrete_sequence=["#ABCD66", "#728944"],
            custom_data=["Category"],
            )
fig.update_traces(marker_line_color='rgb(8,48,107)',
                    marker_line_width=0.1, opacity=0.9,
                    textfont_size=14, textposition="inside",
                    hovertemplate="<br>".join(
                        ["<b>%{y:,.0f}</b> acres of the forested area is", 
                            "in class <i>%{customdata[0]}</i>"]
                    )
                    + "<extra></extra>",
            )
fig.update_layout(title_x=0.5, title_font_size=20,
                    uniformtext_minsize=8, uniformtext_mode='hide',
                    title_font_color='#6680a8',
                    xaxis_title="Seral Stage",
                    yaxis_title="Acres",
                    hovermode="x unified",
                    template=template,
                    legend=dict(title_text='',
                                orientation="h",
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1),
                    legend_title_text='',
                    font_family=font,
                    font_color='#6680a8',
                    plot_bgcolor='white',
                    showlegend=True
                        )
# no legendd
fig.update_traces(showlegend=False)
fig.show()
# save to HTML
fig.write_html(config = {"displayModeBar": False},
               file="DataVisualizations/CompositionAge_Chart_SNRRK.html", 
               full_html= False, 
               include_plotlyjs='cdn')

## Functional Fire

In [ ]:
# get data frame from feature class
df_fire = pd.DataFrame.spatial.from_featureclass("ID_MGMTzones_FunctionalFire_DominantClass_Tahoe_SNVRRK")

df_fire['Name'] = df_fire['Name'].replace(
    to_replace=[
        "Desolation Wilderness",
        "Granite Chief Wilderness",
        "Mt. Rose Wilderness"
    ],
    value="Wilderness"
)

lookup_dict = {
    1: "Low Severity",
    2: "Moderate Severity",
    3: "High Severity"
}
dfMgmt_Fire = df_fire.groupby(['Name', 'gridcode']).agg({'Acres': 'sum'}).reset_index()
dfMgmt_Fire.rename(columns={"gridcode": "Fire Severity", "Acres": "Acres"}, inplace=True)
dfMgmt_Fire['Fire Severity'] = dfMgmt_Fire['Fire Severity'].map(lookup_dict)
dfMgmt_Fire['Percentage'] = ((dfMgmt_Fire['Acres'] / dfMgmt_Fire['Acres'].sum()) * 100).round(1)
dfMgmt_Fire['Acres'] = dfMgmt_Fire['Acres'].round(1)
# filter out Name ''
dfMgmt_Fire = dfMgmt_Fire[dfMgmt_Fire['Name'] != '']
dfMgmt_Fire.to_csv(output / "FireSeverity_ManagementZones.csv", index=False)

In [ ]:
# stacked bar chart of severity by management zone
fig = px.bar(dfMgmt_Fire, x="Name", y="Acres", 
            color="Fire Severity",
            text=dfMgmt_Fire['Percentage'].astype(str) + '%',
            title="Fire Severity Probability by Management Zone",
            labels={"Acres": "Acres", "Name": "Management Zone"},
            color_discrete_sequence=["#91bfdb", "#ffffbf", "#fc8d59"],
            custom_data=["Fire Severity"]
            )
fig.update_traces(marker_line_color='rgb(8,48,107)',
                    marker_line_width=0.1, opacity=0.9,
                    textfont_size=14, textposition="inside",
                    hovertemplate="<br>".join(
                        ["<b>%{y:,.0f}</b> of the forested area is", 
                            "likely to burn at <i>%{customdata[0]}</i>"]
                    )
                    + "<extra></extra>",
            )
fig.update_layout(title_x=0.5, title_font_size=20,
                    uniformtext_minsize=8, uniformtext_mode='hide',
                    title_font_color='#6680a8',
                    xaxis_title="Management Zone",
                    yaxis_title="Acres",
                    hovermode="x unified",
                    template=template,
                    legend=dict(title_text='',
                                orientation="h",
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1),
                    legend_title_text='Fire Severity',
                    font_family=font,
                    font_color='#6680a8',
                    plot_bgcolor='white',
                    showlegend=True
                        )
fig.show()
# save to HTML
fig.write_html(config = {"displayModeBar": False},
               file="DataVisualizations/FireSeverity_ManagementZone.html", 
               full_html= False, 
               include_plotlyjs='cdn')

## Performance Meassures

In [ ]:
# Fuels Treatment Chart
df = get_data_forest_fuel()
plot_forest_fuel(df)